In [51]:
import pandas as pd
import numpy as np
df=pd.read_csv('updated_coswara.csv')

In [52]:
df

,rmse,chroma_stft,spec_cent,spec_bw,rolloff,zcr,mfcc{1},mfcc{2},mfcc{3},mfcc{4},...,rapJitter,ppq5Jitter,ddpJitter,localShimmer,localdbShimmer,apq3Shimmer,aqpq5Shimmer,apq11Shimmer,ddaShimmer,label
0,0.039102,0.378903,786.823461,966.699650,1387.984940,0.043870,-412.08945,126.752335,31.558170,18.483738,...,0.018672,0.021894,0.056015,0.212818,1.734878,0.097995,0.147546,0.206467,0.293984,1
1,0.051812,0.436672,2219.820298,1874.652272,4134.754998,0.209401,-398.93295,50.929253,-17.480385,4.325164,...,0.030713,0.037553,0.092139,0.283908,2.113383,0.137225,0.241707,0.116884,0.411675,0
2,0.010413,0.238371,2002.598308,2058.223470,4324.443295,0.146994,-599.82200,40.048190,6.373952,13.369130,...,0.018672,0.022602,0.056016,0.134015,1.358899,0.061172,0.091838,0.164672,0.183516,1
3,0.078437,0.242433,569.328347,506.956042,979.591497,0.035271,-423.74878,77.112580,-11.768955,6.080464,...,0.050445,0.070875,0.151334,0.181641,1.673087,0.101366,0.093390,0.116884,0.304098,0
4,0.072712,0.327475,1344.446613,1062.582139,2431.292693,0.089481,-327.21740,160.910540,-60.493977,-20.299130,...,0.001202,0.001537,0.003605,0.085836,0.803849,0.035992,0.054323,0.093975,0.107976,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24123,0.056854,0.467262,2845.574993,2044.704050,5269.369989,0.235494,-388.71793,32.990032,-8.879510,20.782476,...,0.023928,0.029485,0.071784,0.160208,1.659819,0.064901,0.137543,0.331425,0.194703,0
24124,0.050149,0.267105,1199.737564,1636.413852,2392.060470,0.059086,-433.10117,58.741127,37.350792,51.354850,...,0.002124,0.002396,0.006373,0.042722,0.419731,0.018084,0.027000,0.041864,0.054251,0
24125,0.000454,0.525523,4996.270042,2579.510698,8270.021928,0.418827,-745.49110,-27.318123,-5.540486,16.303144,...,0.016868,0.019114,0.050605,0.137923,1.255627,0.061327,0.081247,0.116884,0.183981,0
24126,0.031053,0.167581,1890.217497,2748.644250,5175.546000,0.033493,-499.73740,32.764600,46.792103,40.457275,...,0.001794,0.001854,0.005381,0.016609,0.134111,0.007814,0.009204,0.011837,0.023441,1


In [53]:
df = pd.DataFrame(df)

In [54]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers, optimizers
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score


In [72]:
from sklearn.preprocessing import MinMaxScaler
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

scaler = MinMaxScaler()
X = scaler.fit_transform(X)


In [73]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)


In [58]:
model = Sequential()
model.add(GRU(units=32, input_shape=(X_train.shape[1], 1)))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32, verbose=0)

In [61]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
# Evaluate the model on the test set
y_pred = model.predict(X_test)

# Evaluate model on test set
accuracy = accuracy_score(y_test, y_pred.round())
precision = precision_score(y_test, y_pred.round())
recall = recall_score(y_test, y_pred.round())
f1 = f1_score(y_test, y_pred.round())
roc_auc = roc_auc_score(y_test, y_pred)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred.round()).ravel()
specificity = tn / (tn + fp)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC:", roc_auc)
print("Specificity:", specificity)

151/151 [==============================] - 1s 8ms/step
Accuracy: 0.7368421052631579
Precision: 0.7970071567989591
Recall: 0.5611543747136968
F1 Score: 0.6586021505376345
ROC AUC: 0.8156025934936648
Specificity: 0.8819523269012486


In [75]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout



# Define the model
model = Sequential([
    GRU(units=64, return_sequences=True,  input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    GRU(units=64),
    Dropout(0.2),
    Dense(units=1)
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32)







Epoch 1/50
604/604 [==============================] - 55s 74ms/step - loss: 0.7255 - accuracy: 0.5841 - val_loss: 0.6604 - val_accuracy: 0.6119
Epoch 2/50
604/604 [==============================] - 42s 70ms/step - loss: 0.6687 - accuracy: 0.6002 - val_loss: 0.6478 - val_accuracy: 0.6202
Epoch 3/50
604/604 [==============================] - 45s 75ms/step - loss: 0.6703 - accuracy: 0.6020 - val_loss: 0.6294 - val_accuracy: 0.6349
Epoch 4/50
604/604 [==============================] - 45s 74ms/step - loss: 0.6569 - accuracy: 0.6118 - val_loss: 0.6630 - val_accuracy: 0.6022
Epoch 5/50
604/604 [==============================] - 47s 78ms/step - loss: 0.6490 - accuracy: 0.6276 - val_loss: 0.6190 - val_accuracy: 0.6511
Epoch 6/50
604/604 [==============================] - 46s 76ms/step - loss: 0.6368 - accuracy: 0.6457 - val_loss: 0.6142 - val_accuracy: 0.6631
Epoch 7/50
604/604 [==============================] - 45s 75ms/step - loss: 0.6296 - accuracy: 0.6549 - val_loss: 0.6083 - val_accuracy:

In [76]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
# Evaluate the model on the test set
y_pred = model.predict(X_test)

# Evaluate model on test set
accuracy = accuracy_score(y_test, y_pred.round())
precision = precision_score(y_test, y_pred.round())
recall = recall_score(y_test, y_pred.round())
f1 = f1_score(y_test, y_pred.round())
roc_auc = roc_auc_score(y_test, y_pred)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred.round()).ravel()
specificity = tn / (tn + fp)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC:", roc_auc)
print("Specificity:", specificity)

151/151 [==============================] - 5s 25ms/step
Accuracy: 0.7347699958557812
Precision: 0.7318952234206472
Recall: 0.6527714154832799
F1 Score: 0.6900726392251816
ROC AUC: 0.8095024168630818
Specificity: 0.8024971623155505


In [79]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
import numpy as np

# Define the model
model = Sequential([
    GRU(units=64, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    GRU(units=64, return_sequences=True),
    Dropout(0.2),
    GRU(units=64),
    Dropout(0.2),
    Dense(units=1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32)



Epoch 1/50
604/604 [==============================] - 52s 76ms/step - loss: 0.6611 - accuracy: 0.6051 - val_loss: 0.6233 - val_accuracy: 0.6496
Epoch 2/50
604/604 [==============================] - 46s 76ms/step - loss: 0.6272 - accuracy: 0.6517 - val_loss: 0.6144 - val_accuracy: 0.6674
Epoch 3/50
604/604 [==============================] - 59s 98ms/step - loss: 0.6067 - accuracy: 0.6792 - val_loss: 0.5987 - val_accuracy: 0.6768
Epoch 4/50
604/604 [==============================] - 67s 110ms/step - loss: 0.5928 - accuracy: 0.6851 - val_loss: 0.5844 - val_accuracy: 0.6933
Epoch 5/50
604/604 [==============================] - 66s 110ms/step - loss: 0.5796 - accuracy: 0.6953 - val_loss: 0.5606 - val_accuracy: 0.7099
Epoch 6/50
604/604 [==============================] - 68s 113ms/step - loss: 0.5645 - accuracy: 0.7127 - val_loss: 0.5672 - val_accuracy: 0.7014
Epoch 7/50
604/604 [==============================] - 63s 104ms/step - loss: 0.5530 - accuracy: 0.7195 - val_loss: 0.5408 - val_accur

AttributeError: 'Sequential' object has no attribute 'predict_classes'

In [81]:
# Make predictions on test set
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

# Compute evaluation metrics
acc = (y_pred_binary.flatten() == y_test).mean()
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_binary).ravel()
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * precision * recall / (precision + recall)
auc = roc_auc_score(y_test, y_pred)
specificity = tn / (tn + fp)

print(f"Accuracy: {acc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 score: {f1:.4f}")
print(f"AUC: {auc:.4f}")
print(f"Specificity: {specificity:.4f}")


151/151 [==============================] - 5s 36ms/step
Accuracy: 0.7710
Precision: 0.7870
Recall: 0.6770
F1 score: 0.7279
AUC: 0.8451
Specificity: 0.8487


In [84]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, GRU, Dropout
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
import keras.backend as K

def specificity(y_true, y_pred):
    tn = K.sum(K.round(K.clip((1-y_true)*(1-y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1-y_true)*y_pred, 0, 1)))
    specificity = tn / (tn + fp + K.epsilon())
    return specificity

# Define the model
model = Sequential()
model.add(GRU(64, input_shape=(X_train.shape[1], 1), return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(64))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.AUC(name='auc'), specificity])


# Train the model
# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32, verbose=1)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred = np.round(y_pred)
loss, accuracy, precision, recall, auc, specificity = model.evaluate(X_test, y_test, verbose=0)
conf_mat = confusion_matrix(y_test, y_pred)

# Print the results
print('Loss: ', loss)
print('Accuracy: ', accuracy)
print('Precision: ', precision)
print('Recall: ', recall)
print('AUC: ', auc)
print('Specificity: ', specificity)
print('Confusion Matrix: ', conf_mat)


Epoch 1/50
604/604 [==============================] - 295s 455ms/step - loss: 0.6710 - accuracy: 0.5894 - precision_11: 0.5607 - recall_11: 0.3725 - auc: 0.6087 - specificity: 0.7636 - val_loss: 0.6750 - val_accuracy: 0.5535 - val_precision_11: 0.8684 - val_recall_11: 0.0151 - val_auc: 0.6435 - val_specificity: 0.9978
Epoch 2/50
604/604 [==============================] - 271s 448ms/step - loss: 0.6356 - accuracy: 0.6454 - precision_11: 0.6309 - recall_11: 0.4966 - auc: 0.6810 - specificity: 0.7667 - val_loss: 0.6269 - val_accuracy: 0.6540 - val_precision_11: 0.6803 - val_recall_11: 0.4434 - val_auc: 0.7092 - val_specificity: 0.8260
Epoch 3/50
604/604 [==============================] - 375s 621ms/step - loss: 0.6164 - accuracy: 0.6668 - precision_11: 0.6571 - recall_11: 0.5310 - auc: 0.7094 - specificity: 0.7773 - val_loss: 0.5972 - val_accuracy: 0.6815 - val_precision_11: 0.6850 - val_recall_11: 0.5479 - val_auc: 0.7356 - val_specificity: 0.7909
Epoch 4/50
604/604 [====================

604/604 [==============================] - 203s 336ms/step - loss: 0.2771 - accuracy: 0.8783 - precision_11: 0.8891 - recall_11: 0.8311 - auc: 0.9499 - specificity: 0.9162 - val_loss: 0.5370 - val_accuracy: 0.7708 - val_precision_11: 0.7746 - val_recall_11: 0.6958 - val_auc: 0.8507 - val_specificity: 0.8337
Epoch 26: early stopping
151/151 [==============================] - 22s 129ms/step
Loss:  0.5370438098907471
Accuracy:  0.770824670791626
Precision:  0.7746047973632812
Recall:  0.6958314180374146
AUC:  0.8506723642349243
Specificity:  0.8337308168411255
Confusion Matrix:  [[2201  442]
 [ 664 1519]]


In [87]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense, GRU, Dropout, Bidirectional
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score



# Define evaluation metrics
def evaluate(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred)
    spec = recall_score(y_true, y_pred, pos_label=0)
    return acc, prec, rec, f1, roc_auc, spec

# Define GRU model
def create_model():
    model = Sequential()
    model.add(GRU(64, input_shape=(X_train.shape[1], 1), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(GRU(128, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(GRU(256, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(GRU(128, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(GRU(64))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Define number of folds and batch size
n_folds = 3
batch_size = 32

# Initialize lists to store evaluation metrics
accs = []
precs = []
recs = []
f1s = []
roc_aucs = []
specs = []

# Create K-Fold cross-validation object
kf = KFold(n_splits=n_folds, shuffle=True)

# Perform cross-validation
for i, (train, test) in enumerate(kf.split(X)):
    print('Fold %d...' % (i+1))
    X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]
    
    # Create and fit model
    model = create_model()
    model.fit(X_train, y_train, batch_size=batch_size, epochs=5, verbose=1)
    
    # Evaluate model on test set and calculate metrics
    y_pred = model.predict(X_test)
    y_pred_binary = (y_pred >= 0.5).astype(int)
    acc = accuracy_score(y_test, y_pred_binary)
    prec = precision_score(y_test, y_pred_binary)
    rec = recall_score(y_test, y_pred_binary)
    f1 = f1_score(y_test, y_pred_binary)
    roc_auc = roc_auc_score(y_test, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_binary).ravel()
    spec = tn / (tn + fp)  
    
    # Append metrics to lists
    accs.append(acc)
    precs.append(prec)
    recs.append(rec)
    f1s.append(f1)
    roc_aucs.append(roc_auc)
    specs.append(spec)

# Calculate average and standard deviation of metrics across folds
print('Accuracy: %.3f%% +/- %.3f%%' % (np.mean(accs)*100, np.std(accs)*100))
print('Precision: %.3f%% +/- %.3f%%' % (np.mean(precs)*100, np.std(precs)*100))
print('Recall: %.3f%% +/- %.3f%%' % (np.mean(recs)*100, np.std(recs)*100))
print('F1 Score: %.3f%% +/- %.3f%%' % (np.mean(f1s)*100, np.std(f1s)*100))
print('ROC AUC: %.3f%% +/- %.3f%%' % (np.mean(roc_aucs)*100, np.std(roc_aucs)*100))
print('ROC AUC: %.3f%% +/- %.3f%%' % (np.mean(spec)*100, np.std(spec)*100))



Fold 1...
Epoch 1/5
503/503 [==============================] - 167s 310ms/step - loss: 0.6868 - accuracy: 0.5519
Epoch 2/5
503/503 [==============================] - 170s 337ms/step - loss: 0.6611 - accuracy: 0.6015
Epoch 3/5
503/503 [==============================] - 278s 553ms/step - loss: 0.6298 - accuracy: 0.6522
Epoch 4/5
503/503 [==============================] - 282s 561ms/step - loss: 0.6168 - accuracy: 0.6629
Epoch 5/5
252/252 [==============================] - 60s 220ms/step
Fold 2...
Epoch 1/5
503/503 [==============================] - 322s 576ms/step - loss: 0.6868 - accuracy: 0.5531
Epoch 2/5
503/503 [==============================] - 292s 580ms/step - loss: 0.6659 - accuracy: 0.5952
Epoch 3/5
503/503 [==============================] - 163s 323ms/step - loss: 0.6347 - accuracy: 0.6464
Epoch 4/5
503/503 [==============================] - 168s 334ms/step - loss: 0.6257 - accuracy: 0.6558
Epoch 5/5
252/252 [==============================] - 46s 168ms/step


C:\Users\SHANMUKH\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold 3...
Epoch 1/5
503/503 [==============================] - 177s 319ms/step - loss: 0.6824 - accuracy: 0.5645
Epoch 2/5
503/503 [==============================] - 184s 365ms/step - loss: 0.6467 - accuracy: 0.6347
Epoch 3/5
503/503 [==============================] - 191s 379ms/step - loss: 0.6301 - accuracy: 0.6516
Epoch 4/5
503/503 [==============================] - 177s 352ms/step - loss: 0.6176 - accuracy: 0.6622
Epoch 5/5
252/252 [==============================] - 38s 132ms/step
Accuracy: 63.868% +/- 6.133%
Precision: 48.203% +/- 34.087%
Recall: 31.287% +/- 22.147%
F1 Score: 37.942% +/- 26.843%
ROC AUC: 69.525% +/- 5.644%
ROC AUC: 85.380% +/- 0.000%
